In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta, date
from collections import defaultdict
from sklearn import *
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from sklearn.cross_validation import KFold

C:\Users\Kohei\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Kohei\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\Kohei\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [2]:
input_path = "C:/Users/Kohei/Documents/Kaggle/Recruit/00_input/"

### Function

In [3]:
def dummy(df, cats, inplace_): # inplace=True will delete original feature
    for feat in cats:
        print('Creating dummy variables for {}'.format(feat))
        df_dummy = pd.get_dummies(df[feat], drop_first=True, sparse=True)
        df_dummy = df_dummy.rename(columns=lambda x: feat+'_'+ str(x))
        df.drop(([feat]), axis=1, inplace=inplace_)
        df = pd.merge(df, df_dummy, left_index=True, right_index=True)
    return df

In [4]:
lbl = preprocessing.LabelEncoder()

In [5]:
def reserve_calc(df):
    df['visit_datetime'] = pd.to_datetime(df['visit_datetime'])
    df['visit_date'] = df['visit_datetime'].dt.date
    df['visit_hour'] = df['visit_datetime'].dt.hour
    df['visit_date'] = pd.to_datetime(df['visit_date'])
    df['reserve_datetime'] = pd.to_datetime(df['reserve_datetime'])
    df['reserve_date'] = df['reserve_datetime'].dt.date
    df['reserve_date'] = pd.to_datetime(df['reserve_date'])
    df['hr_dif'] = df.apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).seconds/3600, axis=1)
    df['hr_dif_24mod'] = df['hr_dif'] % 24
    return df

### date

In [6]:
df_date = pd.read_csv(os.path.join(input_path, 'date_info.csv')).rename(columns={'calendar_date':'visit_date'})

# date modification and add next days
df_date.drop(['day_of_week'],axis=1, inplace=True)
df_date['visit_date'] = pd.to_datetime(df_date['visit_date'])

# add next and previous days
date1 = [datetime(2017, 6, 1),date(2017, 6, 2)]
holiday1 = [0,0]
add1 = pd.DataFrame({'visit_date':date1,'holiday_flg':holiday1})

date2 = [datetime(2015, 12, 30),date(2015, 12, 31)]
holiday2 = [1,1]
add2 = pd.DataFrame({'visit_date':date2,'holiday_flg':holiday2})

df_date = pd.concat([add2,df_date,add1])
df_date['dow'] = df_date['visit_date'].dt.dayofweek
df_date['month'] = df_date['visit_date'].dt.month

df_date['visit_date+1d'] = (df_date['visit_date'] + timedelta(days=1))


# previos 1-6 days
for d in range(1,2):
    df_date['visit_date-'+str(d)+'d'] = (df_date['visit_date'] - timedelta(days=d*1))


# next day's holiday flag
tmp = df_date.iloc[:,:2]
tmp = tmp.rename(columns={'visit_date':'visit_date+1d','holiday_flg':'holiday_flg+1d'})

df_date = pd.merge(df_date, tmp, how='left', on='visit_date+1d')
df_date['holiday_flg+1d'] = df_date['holiday_flg+1d'].fillna(0)
df_date.drop(['visit_date+1d'],axis=1, inplace=True)

# previous day's holiday flag
tmp = df_date.iloc[:,:2]
tmp = tmp.rename(columns={'visit_date':'visit_date-1d','holiday_flg':'holiday_flg-1d'})

df_date = pd.merge(df_date, tmp, how='left', on='visit_date-1d')
df_date['holiday_flg-1d'] = df_date['holiday_flg-1d'].fillna(0)
df_date.drop(['visit_date-1d'],axis=1, inplace=True)


# revising holidays
# holidays on weekends are not holidays
wkend_holidays = df_date.apply((lambda x:(x.dow=='Sunday' or x.dow=='Saturday') and x.holiday_flg==1), axis=1)
df_date['holiday_flg_rev'] = df_date['holiday_flg']
df_date.loc[wkend_holidays, 'holiday_flg_rev'] = 0

# weight
df_date['weight'] = (df_date.index + 1) / len(df_date)**5

# one-hot-encoding
cats = ['dow','month']
df_date = dummy(df_date, cats, False)
df_date.head(10)

Creating dummy variables for dow
Creating dummy variables for month


,holiday_flg,visit_date,dow,month,holiday_flg+1d,holiday_flg-1d,holiday_flg_rev,weight,dow_1,dow_2,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1,2015-12-30,2,12,1.0,0.0,1,2.605022e-14,0,1,...,0,0,0,0,0,0,0,0,0,1
1,1,2015-12-31,3,12,1.0,1.0,1,5.210044e-14,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,2016-01-01,4,1,1.0,1.0,1,7.815066e-14,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2016-01-02,5,1,1.0,1.0,1,1.042009e-13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2016-01-03,6,1,0.0,1.0,1,1.302511e-13,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,2016-01-04,0,1,0.0,1.0,0,1.563013e-13,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,2016-01-05,1,1,0.0,0.0,0,1.823515e-13,1,0,...,0,0,0,0,0,0,0,0,0,0
7,0,2016-01-06,2,1,0.0,0.0,0,2.084018e-13,0,1,...,0,0,0,0,0,0,0,0,0,0
8,0,2016-01-07,3,1,0.0,0.0,0,2.344520e-13,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,2016-01-08,4,1,0.0,0.0,0,2.605022e-13,0,0,...,0,0,0,0,0,0,0,0,0,0


### visitors

In [7]:
visitors = pd.read_csv(os.path.join(input_path, 'air_visit_data.csv'))
visitors['visit_date'] = pd.to_datetime(visitors['visit_date'])
visitors.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


### air_store

In [8]:
air_str = pd.read_csv(os.path.join(input_path, 'air_store_info_rev.csv'))

air_str['var_max_lat']  = air_str['latitude'].max() - air_str['latitude']
air_str['var_max_long'] = air_str['longitude'].max() - air_str['longitude']

kmeans = KMeans(n_clusters=12, random_state=0).fit(air_str[['latitude','longitude']])
air_str['km_latlong'] = pd.DataFrame(kmeans.predict(air_str[['latitude','longitude']]))


air_str.rename(columns={'air_genre_name':'air_genre'}, inplace=True)
air_str.rename(columns={'air_area_name':'air_area'}, inplace=True)
air_str['air_areaL1'] = air_str['air_area'].apply(lambda x: ' '.join(x.split(' ')[:1]))
air_str['air_areaL2'] = air_str['air_area'].apply(lambda x: ' '.join(x.split(' ')[1]))
air_str['air_areaL3'] = air_str['air_area'].apply(lambda x: ' '.join(x.split(' ')[2]))

air_str['air_genre'] = lbl.fit_transform(air_str['air_genre'])
air_str['air_areaL1_lbl'] = lbl.fit_transform(air_str['air_areaL1'])
air_str['air_areaL2_lbl'] = lbl.fit_transform(air_str['air_areaL2'])
air_str['air_areaL3_lbl'] = lbl.fit_transform(air_str['air_areaL3'])

air_str.drop(['air_area','air_areaL2','air_areaL3'],axis=1, inplace=True)

cats = ['air_areaL3_lbl']
air_str = dummy(air_str, cats, False)
air_str.head()

Creating dummy variables for air_areaL3_lbl


,air_store_id,air_genre,latitude,longitude,var_max_lat,var_max_long,km_latlong,air_areaL1,air_areaL1_lbl,air_areaL2_lbl,...,air_areaL3_lbl_89,air_areaL3_lbl_90,air_areaL3_lbl_91,air_areaL3_lbl_92,air_areaL3_lbl_93,air_areaL3_lbl_94,air_areaL3_lbl_95,air_areaL3_lbl_96,air_areaL3_lbl_97,air_areaL3_lbl_98
0,air_0f0cdeee6c9bf3d7,6,34.695124,135.197852,9.325508,9.075546,0,Hyogo-ken,3,23,...,0,0,0,0,0,0,0,0,0,0
1,air_7cc17a324ae5c7dc,6,34.695124,135.197852,9.325508,9.075546,0,Hyogo-ken,3,23,...,0,0,0,0,0,0,0,0,0,0
2,air_fee8dcf4d619598e,6,34.695124,135.197852,9.325508,9.075546,0,Hyogo-ken,3,23,...,0,0,0,0,0,0,0,0,0,0
3,air_a17f0778617c76e2,6,34.695124,135.197852,9.325508,9.075546,0,Hyogo-ken,3,23,...,0,0,0,0,0,0,0,0,0,0
4,air_83db5aff8f50478e,6,35.658068,139.751599,8.362564,4.521799,1,Tokyo-to,7,29,...,0,0,0,0,0,0,0,0,0,0


### hpg_store

In [9]:
hpg_str = pd.read_csv(os.path.join(input_path, 'hpg_store_info.csv'))

kmeans = KMeans(n_clusters=12, random_state=0).fit(hpg_str[['latitude','longitude']])
hpg_str['km_hpg_latlong'] = pd.DataFrame(kmeans.predict(hpg_str[['latitude','longitude']]))

hpg_str.rename(columns={'hpg_genre_name':'hpg_genre'}, inplace=True)
hpg_str.rename(columns={'hpg_area_name':'hpg_areaL3'}, inplace=True)

hpg_str['hpg_genre'] = lbl.fit_transform(hpg_str['hpg_genre'])

hpg_str.drop(['latitude','longitude'],axis=1, inplace=True)

store = pd.read_csv(os.path.join(input_path, 'store_id_relation.csv'))
hpg_str = pd.merge(hpg_str, store, how='left', on=['hpg_store_id'])
hpg_str.head()

,hpg_store_id,hpg_genre,hpg_areaL3,km_hpg_latlong,air_store_id
0,hpg_6622b62385aec8bf,15,Tōkyō-to Setagaya-ku Taishidō,7,NaN
1,hpg_e9e068dd49c5fa00,15,Tōkyō-to Setagaya-ku Taishidō,7,NaN
2,hpg_2976f7acb4b3a3bc,15,Tōkyō-to Setagaya-ku Taishidō,7,NaN
3,hpg_e51a522e098f024c,15,Tōkyō-to Setagaya-ku Taishidō,7,NaN
4,hpg_e3d0e1519894f275,15,Tōkyō-to Setagaya-ku Taishidō,7,NaN


### air_reserve

In [10]:
air_res = pd.read_csv(os.path.join(input_path, 'air_reserve.csv'))
air_res = reserve_calc(air_res)
air_res['dow'] = air_res['visit_date'].dt.dayofweek

air_res.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,visit_hour,reserve_date,hr_dif,hr_dif_24mod,dow
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016-01-01,19,2016-01-01,3.0,3.0,4
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016-01-01,19,2016-01-01,0.0,0.0,4
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016-01-01,19,2016-01-01,0.0,0.0,4
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016-01-01,20,2016-01-01,4.0,4.0,4
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016-01-01,20,2016-01-01,19.0,19.0,4


### hpg_reserve

In [11]:
hpg_res = pd.read_csv(os.path.join(input_path, 'hpg_reserve.csv'))
hpg_res = reserve_calc(hpg_res)
hpg_res['dow'] = hpg_res['visit_date'].dt.dayofweek

store = pd.read_csv(os.path.join(input_path, 'store_id_relation.csv'))
hpg_res = pd.merge(hpg_res, store, how='left', on=['hpg_store_id'])
hpg_res.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,visit_hour,reserve_date,hr_dif,hr_dif_24mod,dow,air_store_id
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1,2016-01-01,11,2016-01-01,2.0,2.0,4,NaN
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,2016-01-01,13,2016-01-01,7.0,7.0,4,NaN
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2,2016-01-01,16,2016-01-01,2.0,2.0,4,NaN
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5,2016-01-01,17,2016-01-01,6.0,6.0,4,NaN
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13,2016-01-01,17,2016-01-01,14.0,14.0,4,NaN


### Weather

In [12]:
jma = pd.read_csv(os.path.join(input_path, 'jma/jma.csv'))
jma['visit_date'] = pd.to_datetime(jma['date'])
jma.drop('date', axis=1, inplace=True)

jma['weather_daytime'] = lbl.fit_transform(jma['weather_daytime'])
jma['weather_nighttime'] = lbl.fit_transform(jma['weather_nighttime'])

cats = ['weather_daytime','weather_nighttime']
jma = dummy(jma, cats, False)
jma.head()

Creating dummy variables for weather_daytime
Creating dummy variables for weather_nighttime


,prefecture,rainfall,snowfall,wind_max,rainfall_max1h,wind_max_inst,wind_avg,temperature_high,temperature_low,temperature_avg,...,daylight_hr,weather_daytime,weather_nighttime,visit_date,weather_daytime_1,weather_daytime_2,weather_daytime_3,weather_nighttime_1,weather_nighttime_2,weather_nighttime_3
0,Fukuoka-ken,0.0,0,4.1,0.0,8.4,2.1,12.6,3.1,7.3,...,6.6,3,0,2016-01-01,0,0,1,0,0,0
1,Fukuoka-ken,0.0,0,5.8,0.0,9.9,2.2,14.7,8.3,11.6,...,0.6,0,0,2016-01-02,0,0,0,0,0,0
2,Fukuoka-ken,1.0,0,3.1,1.0,4.1,1.3,15.2,8.5,11.6,...,1.4,0,3,2016-01-03,0,0,0,0,0,1
3,Fukuoka-ken,0.0,0,4.7,0.0,8.1,1.8,16.5,7.4,11.0,...,6.7,3,0,2016-01-04,0,0,1,0,0,0
4,Fukuoka-ken,1.0,0,7.0,0.5,11.0,1.5,12.4,8.3,9.9,...,0.0,0,0,2016-01-05,0,0,0,0,0,0


### Train and Test

In [13]:
train = pd.read_csv(os.path.join(input_path, 'air_visit_data.csv'))
train['visit_date'] = pd.to_datetime(train['visit_date'])

test = pd.read_csv(os.path.join(input_path, 'sample_submission.csv'))
test.drop(['visitors'],axis=1, inplace=True)

test['air_store_id'] = test['id'].apply(lambda x: '_'.join(x.split('_')[:2]))
test['visit_date'] = test['id'].apply(lambda x: x.split('_')[-1])
test['visit_date'] = pd.to_datetime(test['visit_date'])
test.drop(['id'],axis=1, inplace=True)

In [15]:
# weighted mean + min,max,count
def visit_numeric(df, tmp, prefix, var, alt_var1, alt_var2):
    wmean = lambda x:((x.weight * x.y).sum()/x.weight.sum())
    df = df.merge(tmp.groupby(var).apply(wmean).reset_index(), on=var, how='left')
    df.rename(columns={0:str(prefix)+'_wmean'}, inplace=True)
    missing = df[str(prefix)+'_wmean'].isnull()
    df.loc[missing,str(prefix)+'_wmean'] = df[missing].merge(tmp.groupby(alt_var1).mean().reset_index(),how='left',on=alt_var1)['y'].values
    missing = df[str(prefix)+'_wmean'].isnull()
    df.loc[missing,str(prefix)+'_wmean'] = df[missing].merge(tmp.groupby(alt_var2).mean().reset_index(),how='left',on=alt_var2)['y'].values
    
    # max
    df.loc[:,str(prefix)+'_max'] = df.merge(tmp.groupby(var).max().reset_index(), on=var, how='left')['y'].values
    missing = df[str(prefix)+'_max'].isnull()
    df.loc[missing,str(prefix)+'_max'] = df[missing].merge(tmp.groupby(alt_var1).max().reset_index(),how='left',on=alt_var1)['y'].values
    missing = df[str(prefix)+'_max'].isnull()
    df.loc[missing,str(prefix)+'_max'] = df[missing].merge(tmp.groupby(alt_var2).max().reset_index(),how='left',on=alt_var2)['y'].values
    
    # min
    df.loc[:,str(prefix)+'_min'] = df.merge(tmp.groupby(var).min().reset_index(), on=var, how='left')['y'].values
    missing = df[str(prefix)+'_min'].isnull()
    df.loc[missing,str(prefix)+'_min'] = df[missing].merge(tmp.groupby(alt_var1).min().reset_index(),how='left',on=alt_var1)['y'].values
    missing = df[str(prefix)+'_min'].isnull()
    df.loc[missing,str(prefix)+'_min'] = df[missing].merge(tmp.groupby(alt_var2).min().reset_index(),how='left',on=alt_var2)['y'].values

    # med
    df.loc[:,str(prefix)+'_med'] = df.merge(tmp.groupby(var).median().reset_index(), on=var, how='left')['y'].values
    missing = df[str(prefix)+'_med'].isnull()
    df.loc[missing,str(prefix)+'_med'] = df[missing].merge(tmp.groupby(alt_var1).median().reset_index(),how='left',on=alt_var1)['y'].values
    missing = df[str(prefix)+'_med'].isnull()
    df.loc[missing,str(prefix)+'_med'] = df[missing].merge(tmp.groupby(alt_var2).median().reset_index(),how='left',on=alt_var2)['y'].values

    # count
    df.loc[:,str(prefix)+'_cnt'] = df.merge(tmp.groupby(var).count().reset_index(), on=var, how='left')['y'].values
    missing = df[str(prefix)+'_cnt'].isnull()
    df.loc[missing,str(prefix)+'_cnt'] = df[missing].merge(tmp.groupby(alt_var1).count().reset_index(),how='left',on=alt_var1)['y'].values
    missing = df[str(prefix)+'_cnt'].isnull()
    df.loc[missing,str(prefix)+'_cnt'] = df[missing].merge(tmp.groupby(alt_var2).count().reset_index(),how='left',on=alt_var2)['y'].values

    return df

In [16]:
def create_data(data_flag, df, begin, end, gap):
    
    # main data
    df_out = df[df.visit_date>=begin]
    df_out = df_out[df_out.visit_date<=end]

    # data used to create features
    df_visitors = visitors[visitors.visit_date<(begin-timedelta(days=gap))]

    # prep data to calculate mean
    df_tmp = pd.merge(df_visitors, df_date, how='left', on=['visit_date'])
    df_tmp = pd.merge(df_tmp, air_str, how='left', on=['air_store_id'])
    df_tmp = df_tmp.rename(columns={'visitors':'y'})
    df_tmp['y'] = np.log1p(df_tmp['y'])

    # prepare main data
    df_out = pd.merge(df_out, df_date, how='left', on=['visit_date'])
    df_out = pd.merge(df_out, air_str, how='left', on=['air_store_id'])
    df_out = pd.merge(df_out, hpg_str[['air_store_id','km_hpg_latlong']], how='left', on=['air_store_id'])

    # days from first date
    df_out['days_from_first_date'] = df_out.apply(lambda r:(r['visit_date']-pd.to_datetime(begin)+timedelta(days=gap+1)).days, axis=1)
    
    # flag
    if data_flag=='train_all':
        df_out['flag'] = 0
    elif data_flag=='train1':
        df_out['flag'] = 1
    elif data_flag=='train2':
        df_out['flag'] = 2
    elif data_flag=='test':
        df_out['flag'] = df_out['days_from_first_date'].apply(lambda x: 1 if x<=6 else 2)
    
    # store x date
    df_out = visit_numeric(df_out, df_tmp, 'dow_all',['air_store_id','dow'],['air_store_id'],['air_store_id'])
    df_out = visit_numeric(df_out, df_tmp, 'dowhol_all',['air_store_id','dow','holiday_flg'],['air_store_id','dow'],['air_store_id'])

    # target encoding
    if data_flag=='test':
        tmp = df_tmp.groupby(['air_genre'])['y'].mean().reset_index().rename(columns={'y':'genre_mean'})
        df_out  = df_out.merge(tmp, on = ['air_genre'], how='left')
        
        tmp = df_tmp.groupby(['air_genre','dow'])['y'].mean().reset_index().rename(columns={'y':'genre_dow_mean'})
        df_out  = df_out.merge(tmp, on = ['air_genre','dow'], how='left')
        
        tmp = df_tmp.groupby(['air_areaL3_lbl'])['y'].mean().reset_index().rename(columns={'y':'areaL3_mean'})
        df_out  = df_out.merge(tmp, on = ['air_areaL3_lbl'], how='left')
        
        tmp = df_tmp.groupby(['air_areaL3_lbl','dow'])['y'].mean().reset_index().rename(columns={'y':'areaL3_dow_mean'})
        df_out  = df_out.merge(tmp, on = ['air_areaL3_lbl','dow'], how='left')
    else:
        kf = KFold(df_out.shape[0], n_folds=5, random_state=1234, shuffle=True)
        for i, (tr_index,vl_index) in enumerate(kf):
            tr, vl = df_out.loc[tr_index].copy(), df_out.loc[vl_index].copy()
            tr['y'] = np.log1p(tr['visitors'])
            tmp = tr.groupby(['air_genre'])['y'].mean().reset_index().rename(columns={'y':'genre_mean'})
            vl  = vl.merge(tmp, on = ['air_genre'], how='left')
            
            tmp = tr.groupby(['air_genre','dow'])['y'].mean().reset_index().rename(columns={'y':'genre_dow_mean'})
            vl  = vl.merge(tmp, on = ['air_genre','dow'], how='left')
            
            tmp = tr.groupby(['air_areaL3_lbl'])['y'].mean().reset_index().rename(columns={'y':'areaL3_mean'})
            vl  = vl.merge(tmp, on = ['air_areaL3_lbl'], how='left')
            
            tmp = tr.groupby(['air_areaL3_lbl','dow'])['y'].mean().reset_index().rename(columns={'y':'areaL3_dow_mean'})
            vl  = vl.merge(tmp, on = ['air_areaL3_lbl','dow'], how='left')
            if i==0:
                tr_all = vl
            else:
                tr_all = pd.concat([tr_all,vl])
        df_out = tr_all
        del tr_all,vl
        
        
    # air_reserve
    df_airres = air_res[air_res.reserve_date<(begin-timedelta(days=gap))]
    df_out.loc[:,'res_ttl'] = df_out.merge(df_airres.groupby(['air_store_id','visit_date']).sum().reset_index(), on=['air_store_id','visit_date'], how='left')['reserve_visitors'].values
    df_out.loc[:,'res_cnt'] = df_out.merge(df_airres.groupby(['air_store_id','visit_date']).count().reset_index(), on=['air_store_id','visit_date'], how='left')['reserve_visitors'].values
    df_out.loc[:,'res_mean'] = df_out.merge(df_airres.groupby(['air_store_id','visit_date']).mean().reset_index(), on=['air_store_id','visit_date'], how='left')['reserve_visitors'].values
    df_out.loc[:,'res_hr_std'] = df_out.merge(df_airres.groupby(['air_store_id','visit_date']).std().reset_index(), on=['air_store_id','visit_date'], how='left')['visit_hour'].values
    
    tmp = df_airres.groupby(['air_store_id','visit_date','dow'])['reserve_visitors'].mean().reset_index().rename(columns={'reserve_visitors':'res_ttl'})
    tmp = tmp.groupby(['air_store_id','dow'])['res_ttl'].mean().reset_index().rename(columns={'res_ttl':'res_ttl_dow_mean'})
    df_out = df_out.merge(tmp, on=['air_store_id','dow'], how='left')
    
    tmp = df_airres.groupby(['air_store_id','visit_date','dow'])['reserve_visitors'].count().reset_index().rename(columns={'reserve_visitors':'res_cnt'})
    tmp = tmp.groupby(['air_store_id','dow'])['res_cnt'].mean().reset_index().rename(columns={'res_cnt':'res_cnt_dow_mean'})
    df_out = df_out.merge(tmp, on=['air_store_id','dow'], how='left')

    # lagged    
    for d in [1,3,5,10,20,30]:
        df_out['visit_date-'+str(d)+'d'] = (df_out['visit_date']-timedelta(days=d*1))
        df_out.loc[:,'lag_'+str(d)+'d'] = df_out.merge(df_tmp[['air_store_id','visit_date','y']],left_on=['air_store_id','visit_date-'+str(d)+'d'],right_on=['air_store_id','visit_date'], how='left')['y'].values
        df_out.drop(['visit_date-'+str(d)+'d'],axis=1, inplace=True)
        
    for w in range(1,21):
        df_out['visit_date-'+str(w)+'w'] = (df_out['visit_date']-timedelta(days=w*7))
        df_out.loc[:,'lag_'+str(w)+'w'] = df_out.merge(df_tmp[['air_store_id','visit_date','y']],left_on=['air_store_id','visit_date-'+str(w)+'w'],right_on=['air_store_id','visit_date'], how='left')['y'].values
        df_out.loc[:,'lag_res_'+str(w)+'w'] = df_out.merge(df_airres.groupby(['air_store_id','visit_date']).sum().reset_index(),left_on=['air_store_id','visit_date-'+str(w)+'w'],right_on=['air_store_id','visit_date'], how='left')['reserve_visitors'].values
        df_out.drop(['visit_date-'+str(w)+'w'],axis=1, inplace=True)
    
    # moving avg
    for d in [5,10,20,30,50,100]:
        tmp = df_tmp[df_tmp.visit_date>=(begin-timedelta(days=gap)-timedelta(days=d))]
        
        tmp2 = tmp.groupby(['air_store_id'])['y'].mean().reset_index().rename(columns={'y':'mean_'+str(d)+'d'})
        tmp2['mean_'+str(d)+'d'] = tmp2['mean_'+str(d)+'d'].fillna(0)
        df_out = pd.merge(df_out, tmp2, on=['air_store_id'], how='left')
        
        tmp2 = tmp.groupby(['air_store_id'])['y'].max().reset_index().rename(columns={'y':'max_'+str(d)+'d'})
        df_out = pd.merge(df_out, tmp2, on=['air_store_id'], how='left')
        
        tmp2 = tmp.groupby(['air_store_id'])['y'].min().reset_index().rename(columns={'y':'min_'+str(d)+'d'})
        df_out = pd.merge(df_out, tmp2, on=['air_store_id'], how='left')
        
        tmp2 = tmp.groupby(['air_store_id'])['y'].std().reset_index().rename(columns={'y':'std_'+str(d)+'d'})
        df_out = pd.merge(df_out, tmp2, on=['air_store_id'], how='left')
        
        df_out['scale_to_maxmin_'+str(d)+'d'] =  (df_out['mean_'+str(d)+'d']-df_out['min_'+str(d)+'d'])/(df_out['max_'+str(d)+'d']-df_out['min_'+str(d)+'d'])        
        df_out['scale_to_std_'+str(d)+'d'] =  (df_out['mean_'+str(d)+'d'])/(df_out['std_'+str(d)+'d'])        

        del tmp, tmp2

    for w in [2,4,8]:
        tmp = df_tmp[df_tmp.visit_date>=(begin-timedelta(days=gap)-timedelta(days=w*7))]
        tmp2 = tmp.groupby(['air_store_id','dow'])['y'].mean().reset_index().rename(columns={'y':'mean_dow_'+str(w)+'w'})
        tmp2['mean_dow_'+str(w)+'w'] = tmp2['mean_dow_'+str(w)+'w'].fillna(0)
        df_out = pd.merge(df_out, tmp2, on=['air_store_id','dow'], how='left')
        
        del tmp, tmp2
        
    # weather
    df_out = df_out.merge(jma, left_on=['air_areaL1','visit_date'], right_on=['prefecture','visit_date'], how='left')
    df_out.drop(['prefecture','air_areaL1'],axis=1, inplace=True)
    df_out.drop(['wind_max_inst','rainfall_max1h','wind_avg','temperature_high','temperature_low','temperature_avg','snowfall_max','humidity_avg','daylight_hr'],axis=1, inplace=True)
    
    # drop
    df_out.drop('weight',axis=1, inplace=True)

    
    # fill na
    df_out['na_cnt'] = df_out.isnull().sum(axis=1)
    df_out = df_out.fillna(-1)
    
    return df_out

In [17]:
# train_all - 39 days split, 0 days gap
ini_date = date(2017,4,23)
df_all = pd.DataFrame()
split_days = 39
skip_days = 39
gap_days = 0
for i in range(1,int(360/skip_days)+1):
    print('%i / %i ' %(i,int(360/skip_days)))
    df_out = create_data('train_all', train,\
                         ini_date-timedelta(days=i*skip_days),\
                         ini_date-timedelta(days=i*skip_days)+timedelta(days=(split_days-1)),\
                         gap_days)
    df_all = pd.concat([df_all,df_out])
df_all.to_csv('train_all.csv', index=False)

1 / 9 
2 / 9 
3 / 9 
4 / 9 
5 / 9 
6 / 9 
7 / 9 
8 / 9 
9 / 9 


In [18]:
# train1 - 6 days split, 0 days gap
ini_date = date(2017,4,23)
df_all = pd.DataFrame()
split_days = 6
skip_days = 6
gap_days = 0
for i in range(1,int(360/skip_days)+1):
    print('%i / %i ' %(i,int(360/skip_days)))
    df_out = create_data('train1', train,\
                         ini_date-timedelta(days=i*skip_days),\
                         ini_date-timedelta(days=i*skip_days)+timedelta(days=(split_days-1)),\
                         gap_days)
    df_all = pd.concat([df_all,df_out])
df_all.to_csv('train1.csv', index=False)

1 / 60 
2 / 60 
3 / 60 
4 / 60 
5 / 60 
6 / 60 
7 / 60 
8 / 60 
9 / 60 
10 / 60 
11 / 60 
12 / 60 
13 / 60 
14 / 60 
15 / 60 
16 / 60 
17 / 60 
18 / 60 
19 / 60 
20 / 60 
21 / 60 
22 / 60 
23 / 60 
24 / 60 
25 / 60 
26 / 60 
27 / 60 
28 / 60 
29 / 60 
30 / 60 
31 / 60 
32 / 60 
33 / 60 
34 / 60 
35 / 60 
36 / 60 
37 / 60 
38 / 60 
39 / 60 
40 / 60 
41 / 60 
42 / 60 
43 / 60 
44 / 60 
45 / 60 
46 / 60 
47 / 60 
48 / 60 
49 / 60 
50 / 60 
51 / 60 
52 / 60 
53 / 60 
54 / 60 
55 / 60 
56 / 60 
57 / 60 
58 / 60 
59 / 60 
60 / 60 


In [19]:
# train2 - 33 days split, 6 days gap
ini_date = date(2017,4,23)
df_all = pd.DataFrame()
split_days = 33
skip_days = 33
gap_days = 6
for i in range(1,int(360/split_days)+1):
    print('%i / %i ' %(i,int(360/skip_days)))
    df_out = create_data('train2', train,\
                         ini_date-timedelta(days=i*skip_days),\
                         ini_date-timedelta(days=i*skip_days)+timedelta(days=(split_days-1)),\
                         gap_days)
    df_all = pd.concat([df_all,df_out])
df_all.to_csv('train2.csv', index=False)

1 / 10 
2 / 10 
3 / 10 
4 / 10 
5 / 10 
6 / 10 
7 / 10 
8 / 10 
9 / 10 
10 / 10 


In [20]:
test_all = create_data('test', test, date(2017,4,23), date(2017,5,31), 0)
test_all.to_csv('test_all.csv', index=False)